# HR Analytics: EDA Summary

**Objective:** Make a concise, insightful, and visually appealing summary. Combine key insights from univariate and bivariate analysis into one polished section.

> ***Note:*** This notebook is a condensed version of hr_analysis.ipynb; please refer to that notebook for a detailed EDA exploration.

### Notebook Sections:
	1. Attrition Overview
	2. Department & Job Role Analysis
	3. Overtime Impact
	4. Age & Tenure Analysis
	5. Monthly Income & Compensation
	6. Gender & Marital Status

In [ ]:
# Setup & Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (10, 6)

from sqlalchemy import create_engine

from dotenv import load_dotenv
import os

print("Libraries loaded successfully")

In [ ]:
# Load Data from PostgreSQL

load_dotenv()  # loads variables from .env

db_host = os.getenv("PG_HOST")
db_port = os.getenv("PG_PORT")
db_name = os.getenv("PG_DB")
db_user = os.getenv("PG_USER")
db_pass = os.getenv("PG_PASS")

conn_str = f"postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"

engine = create_engine(conn_str)
df = pd.read_sql("SELECT * FROM hr_data;", engine)

print("Data loaded successfully")
print("Shape of dataset:", df.shape)
df.head()

### 1. Attrition Overview

In [ ]:
# Attrition counts and percentage
attrition_summary = df['attrition'].value_counts().reset_index()
attrition_summary.columns = ['Attrition', 'Count']
attrition_summary['Percentage'] = round((attrition_summary['Count']/len(df))*100,2)
attrition_summary

**Interpretation (Markdown):**
- Overall attrition rate is 16%.
- Majority of employees stayed; important for retention focus.

**Visualisation:**

In [ ]:
# Set style
sns.reset_defaults()
sns.set(style="whitegrid", palette="Set2")
plt.figure(facecolor='white')

plt.figure(figsize=(5,4))
sns.countplot(x='attrition', hue="attrition", data=df, palette='Set2')
plt.title("Overall Attrition Distribution")
plt.savefig("figures/eda_summary/attrition.png", bbox_inches='tight')
plt.show()

### 2. Department & Job Role Analysis¶

In [ ]:
# Department attrition percentages
dept_attrition = pd.crosstab(df['department'], df['attrition'], normalize='index')*100
dept_attrition

**Interpretation:**
- Sales has highest proportional attrition (~ 20%)
- R&D has largest employee base but lower attrition (~ 14%)
- HR is small but shows ~19% attrition

**Visualisation:**

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x="department", hue="attrition", data=df)
plt.title("Attrition across Departments")
plt.savefig("figures/eda_summary/dept.png", bbox_inches='tight')
plt.show()

In [ ]:
# Job Role attrition percentages
jobrole_attrition = pd.crosstab(df['jobrole'], df['attrition'], normalize='index')*100
jobrole_attrition

**Interpretation:**
- Sales Representatives, Laboratory Technicians and Human Resources have the highest attrition rates (~24–40%).
- Senior/Managerial roles have very low attrition (~2–7%).
- Entry-level and field roles are more likely to leave.

**Visualisation:**

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x='jobrole', hue='attrition', data=df, order=df['jobrole'].value_counts().index)
plt.title("Attrition by Job Role")
plt.xticks(rotation=60)
plt.savefig("figures/eda_summary/jobrole.png", bbox_inches='tight')
plt.show()

### 3. Overtime Impact

In [ ]:
overtime_attrition = pd.crosstab(df['overtime'], df['attrition'], normalize='index')*100
overtime_attrition

**Interpretation:**
- Employees working overtime leave at ~30% vs ~10% for non-overtime.
- Overtime is a key retention factor.

**Visualisation:**

In [ ]:
sns.countplot(x='overtime', hue='attrition', data=df)
plt.title("Attrition vs Overtime")
plt.savefig("figures/eda_summary/overtime_impact.png", bbox_inches='tight')
plt.show()

### 4. Age & Tenure Analysis

In [ ]:
# Create Age group and tenure buckets
df['age_group'] = pd.cut(df['age'], bins=[0,29,45,100], labels=['<30','30-45','45+'])
df['tenure_bucket'] = pd.cut(df['yearsatcompany'], bins=[-1,0,2,5,10,100], labels=['0','1-2','3-5','6-10','10+'])

age_attrition = pd.crosstab(df['age_group'], df['attrition'], normalize='index')*100
tenure_attrition = pd.crosstab(df['tenure_bucket'], df['attrition'], normalize='index')*100
print(age_attrition)
print(tenure_attrition)

**Interpretation:**
- <30 years: highest attrition (~27–30%)
- 30–45 years: attrition drops (~12–14%)
- 0–2 years tenure: ~29–36% attrition
- Employees with 10+ years: least likely to leave (~8%)

**Visualisation:**

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))
sns.countplot(x='age_group', hue='attrition', data=df, ax=axes[0])
axes[0].set_title("Attrition by Age Group")
sns.countplot(x='tenure_bucket', hue='attrition', data=df, ax=axes[1])
axes[1].set_title("Attrition by Tenure Buckets")
plt.savefig("figures/eda_summary/age_tenure_analysis.png", bbox_inches='tight')
plt.show()

### 5. Monthly Income & Compensation

In [ ]:
income_attrition = pd.crosstab(df['jobrole'], df['attrition'], normalize='index')*100
income_attrition

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='attrition', y='monthlyincome', data=df)
plt.title("Monthly Income vs Attrition")
plt.savefig("figures/eda_summary/monthlyincome_vs_attrition.png", bbox_inches='tight')
plt.show()

**Interpretation:**
- Employees who left earn lower median monthly income (`~$4.8k`) than those who stayed (`~$6.8k`).
- Compensation is likely a retention factor.

### 6. Gender & Marital Status

In [ ]:
gender_attrition = pd.crosstab(df['gender'], df['attrition'], normalize='index')*100
maritalstatus_attrition = pd.crosstab(df['maritalstatus'], df['attrition'], normalize='index')*100
print(gender_attrition)
print(maritalstatus_attrition)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))
sns.countplot(x='gender', hue='attrition', data=df, ax=axes[0])
axes[0].set_title("Attrition by Gender")
sns.countplot(x='maritalstatus', hue='attrition', data=df, ax=axes[1])
axes[1].set_title("Attrition by Marital Status")
plt.savefig("figures/eda_summary/attrition_gender_maritalstatus.png", bbox_inches='tight')
plt.show()

**Interpretation:**
- Gender has minor impact on attrition.
- Single employees leave more frequently (`~19%`) than married (`~14%`) or divorced (`~12%`).

### Summary
- **Attrition Distribution:** Majority of employees (`84%`) stayed, while `16%` left.
- **Department:**
  * Sales has highest proportional attrition (`~20%`)
  *  R&D has largest employee base but lower attrition (`~14%`)
  *  HR is small but shows `~19%` attrition.
- **Overtime:** Employees working overtime leave at `~30%` vs `~10%` for non-overtime; making overtime a key retention factor.
- **Job Role:**
  * Sales Representatives, Laboratory Technicians and Human Resources have the highest attrition rates (~24–40%).
  * Senior/Managerial roles have very low attrition (`~2–7%`).
- **Age:** Employees with age less than 30 years face highest attrition (`~27–30%`). While, attrition rate drops (`~12–14%`) for 30–45 years age bracket.
- **Tenure:** Employees with 0 to 2 years of tenure have highest attrition rate (`~29–36%`), while employees with 10+ years at company are least likely to leave (`~8%`).
- **Monthly Income & Compensation:** Employees who left earn lower median monthly income (`~$4.8k`) than those who stayed (`~$6.8k`). Compensation is likely a retention factor.
- **Gender & Marital Status:**
  * Gender has minor impact on attrition.
  * Single employees leave more frequently compared to married or divorced employees.